# Import Libraries

In [1]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
import torchvision
import sys, os

import model
from utils import plot_images
from torch.optim.lr_scheduler import StepLR
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings('ignore')

In [2]:
### this is for running in local ###
try:
    os.environ['HTTP_PROXY']='http://185.46.212.90:80'
    os.environ['HTTPS_PROXY']='http://185.46.212.90:80'
    print ("proxy_exported")
except:
    None

proxy_exported


## Data Transformations

We first start with defining our data transformations. We need to think what our data is and how can we augment it to correct represent images which it might not see otherwise.


In [3]:
# # Train Phase transformations

train_transforms = transforms.Compose([
                                      #transforms.RandomApply([transforms.CenterCrop(22), ], p=0.1),
                                       #transforms.RandomApply([transforms.CenterCrop(22), ], p=0.1),
                                       #transforms.Resize((28, 28)),
                                       transforms.RandomAutocontrast(p=0.1),
                                       transforms.RandomRotation((-7., 7.), fill=1),
                                       transforms.ToTensor(),
                                       transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
                                    ])
                            
        
# Test Phase transformations
test_transforms = transforms.Compose([
                                      #  transforms.Resize((28, 28)),
                                      #  transforms.ColorJitter(brightness=0.10, contrast=0.1, saturation=0.10, hue=0.1),
                                       transforms.ToTensor(),
                                       transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
                                       ])


# Test Phase transformations
inv_transforms = transforms.Compose([
                                      #  transforms.Resize((28, 28)),
                                      #  transforms.ColorJitter(brightness=0.10, contrast=0.1, saturation=0.10, hue=0.1),
                                       transforms.ToTensor(),
                                       transforms.Normalize(mean=[-0.485/0.229, -0.456/0.224, -0.406/0.255],
                                                            std=[1/0.229, 1/0.224, 1/0.255]),
                                       ])


# Dataset and Creating Train/Test Split

In [4]:
train = datasets.CIFAR10('./data', train=True, download=True, transform=train_transforms)
test = datasets.CIFAR10('./data', train=False, download=True, transform=test_transforms)
print (train)
print (test)

Files already downloaded and verified
Files already downloaded and verified
Dataset CIFAR10
    Number of datapoints: 50000
    Root location: ./data
    Split: Train
    StandardTransform
Transform: Compose(
               RandomAutocontrast(p=0.1)
               RandomRotation(degrees=[-7.0, 7.0], interpolation=nearest, expand=False, fill=1)
               ToTensor()
               Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
           )
Dataset CIFAR10
    Number of datapoints: 10000
    Root location: ./data
    Split: Test
    StandardTransform
Transform: Compose(
               ToTensor()
               Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
           )


# Dataloader Arguments & Test/Train Dataloaders


In [5]:
SEED = 1

# CUDA?
cuda = torch.cuda.is_available()
print("CUDA Available?", cuda)

# For reproducibility
torch.manual_seed(SEED)

if cuda:
    torch.cuda.manual_seed(SEED)

# dataloader arguments - something you'll fetch these from cmdprmt
dataloader_args = dict(shuffle=True, batch_size=64, num_workers=4, pin_memory=True) if cuda else dict(shuffle=True, batch_size=64)

# train dataloader
train_loader = torch.utils.data.DataLoader(train, **dataloader_args)

# test dataloader
test_loader = torch.utils.data.DataLoader(test, **dataloader_args)
print (len(train_loader))
print (len(test_loader))

CUDA Available? True
782
157


In [6]:
print ("TRAIN DATASET")
batch_data_train, batch_label_train = next(iter(train_loader)) 
#figure_train = plot_images(batch_data_train, batch_label_train.tolist(), 12, 3, 'CIFAR10')

TRAIN DATASET


In [7]:
print ('TEST DATATEST')
batch_data_test, batch_label_test = next(iter(test_loader)) 
# figure_test = plot_images(batch_data_test, batch_label_test.tolist(), 20, 4, 'CIFAR10')

TEST DATATEST


In [8]:
from torchsummary import summary
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
print(device)

cuda


# Training and Testing

Looking at logs can be boring, so we'll introduce **tqdm** progressbar to get cooler logs.

Let's write train and test functions

In [9]:
from tqdm import tqdm

train_losses = []
test_losses = []
train_acc = []
test_acc = []

def train(model, device, train_loader, optimizer, epoch):
    model.train()
    pbar = tqdm(train_loader)
    correct = 0
    processed = 0
    for batch_idx, (data, target) in enumerate(pbar):
        # get samples
        data, target = data.to(device), target.to(device)

        # Init
        optimizer.zero_grad()
        # In PyTorch, we need to set the gradients to zero before starting to do backpropragation because PyTorch accumulates the gradients on subsequent backward passes.
        # Because of this, when you start your training loop, ideally you should zero out the gradients so that you do the parameter update correctly.

        # Predict
        y_pred = model(data)

        # Calculate loss
        loss = F.nll_loss(y_pred, target)
        train_losses.append(loss)

        # Backpropagation
        loss.backward()
        optimizer.step()

        # Update pbar-tqdm

        pred = y_pred.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
        correct += pred.eq(target.view_as(pred)).sum().item()
        processed += len(data)

        #pbar.set_description(desc= f'Loss={loss.item()} Batch_id={batch_idx} Accuracy={100*correct/processed:0.2f}')
        train_acc.append(100*correct/processed)
        
    print (" TRAIN ACCURACY: ",100*correct/processed, "TRAIN LOSS: ", loss.item()),

def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            

            
            
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            
            
            for i in pred.tolist().values():
                print (i)
                #if (pred.eq(target.view_as(pred)) == False):
                #    print (i)
            
            print (pred.eq(target.view_as(pred)))
#             if (pred.eq(target.view_as(pred)) ==  False):
#                 print ()
            print (target)
            print (pred.tolist())
            print (type(pred))
            transfom = pred.ToPILImage()
            pred_new = inv_transforms(transfom)
            
            print ("***********pred_new**********")
            print (pred_new)
                      
            
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)
    test_losses.append(test_loss)

    #print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.2f}%)\n'.format(
     #   test_loss, correct, len(test_loader.dataset),
      #  100. * correct / len(test_loader.dataset)))

    test_acc.append(100. * correct / len(test_loader.dataset))
    print (" TEST ACCURACY: ",100*correct/len(test_loader.dataset), "TEST LOSS: ", test_loss)
    
def plot_losses(train_losses, train_acc, test_losses, test_acc):
    t = [t_items.item() for t_items in train_losses]
    %matplotlib inline
    
    fig, axs = plt.subplots(2,2,figsize=(15,10))
    axs[0, 0].plot(t)
    axs[0, 0].set_title("Training Loss")
    axs[1, 0].plot(train_acc[4000:])
    axs[1, 0].set_title("Training Accuracy")
    axs[0, 1].plot(test_losses)
    axs[0, 1].set_title("Test Loss")
    axs[1, 1].plot(test_acc)
    axs[1, 1].set_title("Test Accuracy")

# Model Params for Batch Normalization
Can't emphasize on how important viewing Model Summary is.
Unfortunately, there is no in-built model visualizer, so we have to take external help

In [10]:
m = model.Net_batch_normalization().to(device)
summary(m, input_size=(3, 32, 32))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 48, 32, 32]           1,344
              ReLU-2           [-1, 48, 32, 32]               0
       BatchNorm2d-3           [-1, 48, 32, 32]              96
         Dropout2d-4           [-1, 48, 32, 32]               0
            Conv2d-5           [-1, 48, 32, 32]          20,784
              ReLU-6           [-1, 48, 32, 32]               0
       BatchNorm2d-7           [-1, 48, 32, 32]              96
         Dropout2d-8           [-1, 48, 32, 32]               0
            Conv2d-9           [-1, 32, 32, 32]           1,568
        MaxPool2d-10           [-1, 32, 16, 16]               0
           Conv2d-11           [-1, 32, 16, 16]           9,248
             ReLU-12           [-1, 32, 16, 16]               0
      BatchNorm2d-13           [-1, 32, 16, 16]              64
        Dropout2d-14           [-1, 32,

In [11]:
optimizer = optim.SGD(m.parameters(), lr=0.01, momentum=0.9)
scheduler = StepLR(optimizer, step_size=4, gamma=0.1, verbose=False)

EPOCHS = 20
for epoch in range(EPOCHS):
    print("EPOCH:", epoch),
    train(m, device, train_loader, optimizer, epoch)
    scheduler.step()
    test(m, device, test_loader)

EPOCH: 0


100% 782/782 [00:04<00:00, 168.92it/s]

 TRAIN ACCURACY:  41.752 TRAIN LOSS:  1.5383192300796509


AttributeError: 'list' object has no attribute 'values'

# Let's Train and test our model

This time let's add a scheduler for out LR.

In [ ]:
plot_losses(train_losses, train_acc, test_losses, test_acc)

**********************************************************

# Model Params for Group Normalization
Can't emphasize on how important viewing Model Summary is.
Unfortunately, there is no in-built model visualizer, so we have to take external help

In [ ]:
m = model.Net_group_norm().to(device)
summary(m, input_size=(3, 32, 32))

In [ ]:
optimizer = optim.SGD(m.parameters(), lr=0.01, momentum=0.9)
scheduler = StepLR(optimizer, step_size=4, gamma=0.1, verbose=False)

EPOCHS = 20
for epoch in range(EPOCHS):
    print("EPOCH:", epoch)
    train(m, device, train_loader, optimizer, epoch)
    scheduler.step()
    test(m, device, test_loader)

In [ ]:
plot_losses(train_losses, train_acc, test_losses, test_acc)

# Model Params for Layer Normalization
Can't emphasize on how important viewing Model Summary is.
Unfortunately, there is no in-built model visualizer, so we have to take external help

In [ ]:
m = model.Net_layer_normalization().to(device)
summary(m, input_size=(3, 32, 32))

In [ ]:
optimizer = optim.SGD(m.parameters(), lr=0.01, momentum=0.9)
scheduler = StepLR(optimizer, step_size=4, gamma=0.1, verbose=False)

EPOCHS = 20
for epoch in range(EPOCHS):
    print("EPOCH:", epoch)
    train(m, device, train_loader, optimizer, epoch)
    scheduler.step()
    test(m, device, test_loader)

In [ ]:
plot_losses(train_losses, train_acc, test_losses, test_acc)